In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import os
import pandas as pd
from pykonal import EikonalSolver, LinearInterpolator3D, Grid3D
from pykonal import transform
import seispy

EARTH_RADIUS = 6371.
GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']
VMODEL_PATH  = os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'White_et_al_2019a', 'White_et_al_2019a.regular.npz')
DB_PATH      = os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'events', 'malcolmw', 'SJFZ_catalog_2008-2016.h5')

In [2]:
%matplotlib widget

Define convenience class to compute traveltimes using a source-centered coordinate system in the near field.

In [24]:
class TwoStageSolver(object):
    
    def __init__(self, coord_sys='spherical'):
        '''
        A convenience class to compute traveltimes using a
        source-centered coordinate system in the near field.
        '''
        self.coord_sys = coord_sys


    @property
    def near_field(self):
        '''
        The source-centered EikonalSolver for the near field.
        '''
        if not hasattr(self, '_near_field'):
            self._near_field = EikonalSolver(coord_sys='spherical')
        return (self._near_field)
    
    @property
    def far_field(self):
        '''
        The EikonalSolver for the far field.
        '''
        if not hasattr(self, '_far_field'):
            self._far_field = EikonalSolver(coord_sys=self.coord_sys)
        return (self._far_field)
    
    @property
    def src_loc(self):
        '''
        The coordinates of the source.
        '''
        return (self._src_loc)
    
    @src_loc.setter
    def src_loc(self, value):
        value = np.array(value)
        if np.any(value < self.far_field.vgrid.min_coords)\
                or np.any(value > self.far_field.vgrid.max_coords):
            raise(ValueError('Source location must lie inside far-field grid.'))
        self._src_loc = value

    def solve(self):
        self._init_near_field()
        self.near_field.solve()
        self.far_field.transfer_travel_times_from(
            self.near_field,
            self.src_loc  * [-1, 1, 1],
            set_alive=True
        )
        self.far_field.solve()
    
    def _init_near_field(self):
        if self.coord_sys == 'cartesian':
            drho = np.min(self.far_field.pgrid.min_coords) / 5
        else:
            drho = self.far_field.pgrid.node_intervals[0] / 5
        self.near_field.vgrid.min_coords     = drho, 0, 0
        self.near_field.vgrid.node_intervals = drho, np.pi/20, np.pi/20
        self.near_field.vgrid.npts           = 100, 21, 40
        self.near_field.transfer_velocity_from(self.far_field, self.src_loc)
        for it in range(self.near_field.vgrid.npts[1]):
            for ip in range(self.near_field.vgrid.npts[2]):
                idx = (0, it, ip)
                self.near_field.uu[idx]     = drho / self.near_field.vvp[idx]
                self.near_field.is_far[idx] = False
                self.near_field.close.push(*idx)

# Synthetic data

This example tests the location algorithm for the simplest and most ideal case: Homogeneous velocity structure and station coverage

## Generate synthetic data

### Define the computational grid

In [52]:
TAG            = 'homogeneous_0.0'
lat0, lon0, z0 = 44, 44, -1     # homogeneous_0.0
dlat, dlon, dz = 0.01, 0.01, 1 # homogeneous_0.0
nlat, nlon, nz = 201, 201, 26  # homogeneous_0.0
latmax         = lat0 + (nlat-1)*dlat
lonmax         = lon0 + (nlon-1)*dlon
zmax           = z0 + (nz-1)*dz

rho0, theta0, phi0 = EARTH_RADIUS-zmax, np.pi/2-np.radians(latmax), np.radians(lon0)
drho, dtheta, dphi = dz, np.radians(dlat), np.radians(dlon)
nrho, ntheta, nphi = nz, nlat, nlon

### Define the station locations

In [53]:
def init_stations(lat0, latmax, lon0, lonmax):
    stations = pd.DataFrame(columns=['sta_code', 'lat', 'lon', 'depth'])
    dlat, dlon = 0.25, 0.25 # homogeneous_0.0
    ilat = 0
    lat  = lat0
    while lat <= latmax:
        ilon, lon  = 0, lon0
        while lon <= lonmax:
            sta_code = f'{chr(ord("A") + ilat//26)}{chr(ord("A") + ilat%26)}{ilon:02d}'
            stations = stations.append(
                pd.DataFrame(
                    [[sta_code, lat, lon, 0]],
                    columns=['sta_code', 'lat', 'lon', 'depth']
                ),
                ignore_index=True
            )
            ilon += 1
            lon  += dlon
        ilat += 1
        lat  += dlat
    return(stations)

def init_events(lat0, latmax, lon0, lonmax, z0, zmax):
    events         = pd.DataFrame(columns=['lat', 'lon', 'depth', 'event_id'])
    lat_avg        = (lat0+latmax) / 2
    lon_avg        = (lon0+lonmax) / 2
    z_avg          = (z0+zmax) / 2
    dlat, dlon, dz = 0.1, 0.1, 1 # homogeneous_0.0
    ilat           = 0
    lat, lon, z    = lat0, lon0, z0
    while lat <= latmax:
        events = events.append(
                pd.DataFrame(
                    [[lat, lon_avg, z_avg, -1]],
                    columns=['lat', 'lon', 'depth', 'event_id']
                ),
                ignore_index=True
        )
        lat += dlat
    while lon <= lonmax:
        events = events.append(
                pd.DataFrame(
                    [[lat_avg, lon, z_avg, -1]],
                    columns=['lat', 'lon', 'depth', 'event_id']
                ),
                ignore_index=True
        )
        lon += dlon
    while z <= zmax:
        events = events.append(
                pd.DataFrame(
                    [[lat_avg, lon_avg, z, -1]],
                    columns=['lat', 'lon', 'depth', 'event_id']
                ),
                ignore_index=True
        )
        z += dz
    events['event_id'] = events.index.values
    return (events)

stations = init_stations(lat0, latmax, lon0, lonmax)
events = init_events(lat0, latmax, lon0, lonmax, z0, zmax)

In [55]:
# event   = events.iloc[52]
# station = stations.iloc[39]
event   = events.iloc[np.random.randint(0, len(events))]
station = stations.iloc[np.random.randint(0, len(stations))]
print(event, station)

lat         44.8
lon           45
depth       11.5
event_id       8
Name: 8, dtype: object sta_code     AB05
lat         44.25
lon         45.25
depth           0
Name: 14, dtype: object


In [56]:
# Event
event_solver = TwoStageSolver(coord_sys='spherical')
event_solver.far_field.vgrid.min_coords      = rho0, theta0, phi0
event_solver.far_field.vgrid.node_intervals  = drho, dtheta, dphi
event_solver.far_field.vgrid.npts            = nrho, ntheta, nphi
event_solver.far_field.vv                    = np.ones(event_solver.far_field.vgrid.npts)
event_solver.src_loc                         = seispy.coords.as_geographic(
    event[['lat', 'lon', 'depth']]
).to_spherical()

%time event_solver.solve()
uui_es = LinearInterpolator3D(event_solver.far_field.pgrid, event_solver.far_field.uu)

# event_solver._init_near_field()
# station_solver.near_field.solve()

CPU times: user 10.8 s, sys: 189 ms, total: 10.9 s
Wall time: 10.9 s


In [57]:
station_solver = TwoStageSolver(coord_sys='spherical')
station_solver.far_field.vgrid.min_coords      = rho0, theta0, phi0
station_solver.far_field.vgrid.node_intervals  = drho, dtheta, dphi
station_solver.far_field.vgrid.npts            = nrho, ntheta, nphi
station_solver.far_field.vv                    = np.ones(station_solver.far_field.vgrid.npts)
station_solver.src_loc                         = seispy.coords.as_geographic(
    station[['lat', 'lon', 'depth']]
).to_spherical()

%time station_solver.solve()
uui_se = LinearInterpolator3D(station_solver.far_field.pgrid, station_solver.far_field.uu)

# station_solver._init_near_field()
# station_solver.near_field.solve()

CPU times: user 12 s, sys: 205 ms, total: 12.2 s
Wall time: 12.1 s


In [58]:
(
    uui_es(seispy.coords.as_geographic(station[['lat', 'lon', 'depth']]).to_spherical()),
    uui_se(seispy.coords.as_geographic(event[['lat', 'lon', 'depth']]).to_spherical())
)

(65.24345544856102, 65.24009319582828)

In [59]:
solver = event_solver

nodes = seispy.coords.as_spherical(solver.far_field.pgrid.nodes).to_cartesian()
src   = seispy.coords.as_spherical(solver.src_loc).to_cartesian()
err = np.sqrt(np.sum(np.square(nodes-src), axis=-1)) - solver.far_field.uu
nodes = nodes.rotate(phi0 + dphi*nphi/2, theta0 + dtheta*ntheta/2, 0)

event_nf   = seispy.coords.as_spherical(
    transform.sph2sph(
        solver.near_field.pgrid.nodes, 
        solver.src_loc
    )
).to_cartesian(
).rotate(
   phi0 + dphi*nphi/2, theta0 + dtheta*ntheta/2, 0
)

station_nf   = seispy.coords.as_spherical(
    transform.sph2sph(
        station_solver.near_field.pgrid.nodes, 
        station_solver.src_loc
    )
).to_cartesian(
).rotate(
   phi0 + dphi*nphi/2, theta0 + dtheta*ntheta/2, 0
)


plt.close('all')
fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1, projection='3d')
i1, i2, i3 = slice(None,None,2), slice(None,None,2), slice(None,None,2)
ax.scatter(
    nodes[i1,i2,i3,0].flatten(), 
    nodes[i1,i2,i3,1].flatten(),
    nodes[i1,i2,i3,2].flatten(),
    s=1
)
ax.scatter(
    event_nf[i1,i2,i3,0].flatten(), 
    event_nf[i1,i2,i3,1].flatten(),
    event_nf[i1,i2,i3,2].flatten(),
    s=1
)
ax.scatter(
    station_nf[i1,i2,i3,0].flatten(), 
    station_nf[i1,i2,i3,1].flatten(),
    station_nf[i1,i2,i3,2].flatten(),
    s=1
)
ax.set_zlim(6261, 6461)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(6261, 6461)

# END TEST

### Plot traveltime field

In [ ]:
nodes   = solver_p.near_field.pgrid.mesh
xx_near = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
yy_near = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
zz_near = nodes[...,0] * np.cos(nodes[...,1])

nodes   = solver_p.far_field.pgrid.mesh
xx_far  = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
yy_far  = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
zz_far  = nodes[...,0] * np.cos(nodes[...,1])


plt.close('all')
fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(
#     xx_far.flatten(), yy_far.flatten(), zz_far.flatten(),
#     c=solver_p.far_field.uu.flatten(),
#     cmap=plt.get_cmap('jet_r')
# )
ax.scatter(
    xx_near.flatten(), yy_near.flatten(), zz_near.flatten(),
    c=solver_p.near_field.uu.flatten(),
    cmap=plt.get_cmap('jet_r'),
    s=1
)

# Real data

Read velocity model

In [ ]:
vmod = seispy.velocity.VelocityModel(VMODEL_PATH, fmt='npz')

Read network geometry

In [ ]:
db = seispy.pandas.catalog.Catalog(DB_PATH, fmt='hdf5', tables=['site'])

In [ ]:
solver_p = TwoStageSolver()
solver_p.far_field.vgrid.min_coords      = vmod.rho0, vmod.theta0, vmod.phi0
solver_p.far_field.vgrid.node_intervals  = vmod.drho, vmod.dtheta, vmod.dphi
solver_p.far_field.vgrid.npts            = vmod.nrho, vmod.ntheta, vmod.nphi
solver_p.far_field.vv                    = vmod._vp

In [ ]:
%time solver_p.solve()

In [ ]:
np.savez('/Users/malcolmwhite/Desktop/test.npz', uu=solver_p.far_field.uu)

In [ ]:
ir, it, ip = slice(None), slice(None), 0

In [ ]:
ir, it, ip = slice(None), slice(None), 0

nodes = solver_p.far_field.pgrid.mesh
nodes_x = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
nodes_y = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])
nodes_z = nodes[...,0] * np.cos(nodes[...,1])

fig = plt.figure()
ax  = fig.add_subplot(1, 1, 1)
ax.pcolormesh(
    nodes_y[ir, it, ip],
    nodes_z[ir, it, ip],
    solver_p.far_field.uu[ir, it, ip],
    cmap=plt.get_cmap('jet_r'),
    shading='gouraud'
)

In [ ]:
solver_p.far_field.uu[ir, it, ip].shape

In [ ]:
%time solver_p.near_field.solve()

In [ ]:
solver_p.near_field.uu

In [ ]:
solver_p.src_loc = 6365, 0.98, -2.03

In [ ]:
solver_p.src_loc